---
layout: post
title: "「爬虫」豆瓣电影封面海报爬取(下)"
subtitle: 'Python Web 爬虫'
author: "Hufe"
header-img: "img/post-bg-python.jpg"
header-mask: 0.3
mathjax: true
tags:
  - Python
  - 爬虫
---

> 参考自：[极客时间 数据分析实战45讲 10丨Python爬虫：如何自动化下载王祖贤海报?](https://time.geekbang.org/column/article/76001)  
>
>
> [崔庆才微信公众号  进击的Coder 别只用 Selenium，新神器 Pyppeteer 绕过淘宝更简单！](https://mp.weixin.qq.com/s?__biz=MzIzNzA4NDk3Nw==&mid=2457737358&idx=1&sn=fb88904cac67300130cabbc72bc4a650&chksm=ff44b0d0c83339c6496cabf8e09e8a9e0316df1032ef7523ba6ab7f4f6a4bea1cd4c02eb7d7b&mpshare=1&scene=1&srcid=&key=78b2720e78341aa28774612abcab5307b4cdc9ad1b5d1267c04ee8888a00fe5db69c9a97d040ca8b21302cb3db3ca9ca8f34cc771fde539311540e95369cc90e826f382e4ca3f7e63d92232c425f136a&ascene=1&uin=NzY5MzM5MTAy&devicetype=Windows+10&version=62060739&lang=zh_CN&pass_ticket=bQbopgzSyMCKUcltN1KlYrRMsOkEXodvWIuVZnvFsf26wtcmJtf9Ljz3EkaIyuI8)

上篇是通过 JSON 数据爬取，本篇是通过 selenium + webdriver+ XPath 定位爬取。

本次爬取的页面是`https://movie.douban.com/subject_search?search_text=王祖贤&cat=1002"`  
这个页面是 JavaScript 渲染而成的，用基本的 requests 库请求得到的 HTML 结果里面是不包含页面中所见的条目内容的。
此类页面可通过模拟浏览器执行，此种情形适用于网页接口和逻辑较为复杂的情况，可以直接以**可见即可爬**的方式进行爬取，如可以使用 Selenium、Splinter、Spynner、pyppeteer、PhantomJS、Splash、requests-html 等来实现。  
本篇使用的是Pyppeteer 和 Selenium

## Pyppeteer

In [10]:
import requests
from pyquery import PyQuery as pq

url = 'http://quotes.toscrape.com/js/'
response = requests.get(url)
doc = pq(response.text)
print('Quotes:', doc('.quote').length)

Quotes: 0


- 安装：
`pip install pyppeteer`

在 Pyppetter 中，实际上它背后也是有一个类似 Chrome 浏览器的 Chromium 浏览器在执行一些动作进行网页渲染，具体内容请看崔大的文章  
所以，第一次使用pyppeteer，会自动安装chromium
```
[W:pyppeteer.chromium_downloader] start chromium download.
Download may take a few minutes.
100%|███████████████████████████████████████████████████████████████| 133194757/133194757 [00:25<00:00, 5263685.65it/s]
[W:pyppeteer.chromium_downloader] 
chromium download done.
[W:pyppeteer.chromium_downloader] chromium extracted to: C:\Users\huife\AppData\Local\pyppeteer\pyppeteer\local-chromium\575458and
```

In [ ]:
import asyncio
from pyppeteer import launch
from pyquery import PyQuery as pq

async def main():
    browser = await launch()
    page = await browser.newPage()
    await page.goto('http://quotes.toscrape.com/js/')
    doc = pq(await page.content())
    print('Quotes:', doc('.quote').length)
    await browser.close()

asyncio.get_event_loop().run_until_complete(main())

```
Quotes: 10
```

![title](https://raw.githubusercontent.com/hufe09/GitNote-Images/master/gitnote/2019/05/14/1557845283427-1557845284044.png)
```
<div class="quote">
    <span class="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
    <span>by<small class="author">Albert Einstein</small></span>
    <div class="tags">Tags:
        <a class="tag">change</a>
        <a class="tag">deep-thoughts</a>
        <a class="tag">thinking</a>
        <a class="tag">world</a>
    </div>
</div>
```
```
<script>
    var data = [
    {
        "tags": [
            "change",
            "deep-thoughts",
            "thinking",
            "world"
        ],
        "author": {
            "name": "Albert Einstein",
            "goodreads_link": "/author/show/9810.Albert_Einstein",
            "slug": "Albert-Einstein"
        },
        "text": "\u201cThe world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.\u201d"
    },
    {
        "tags": [
            "abilities",
            "choices"
        ],
        "author": {
            "name": "J.K. Rowling",
            "goodreads_link": "/author/show/1077326.J_K_Rowling",
            "slug": "J-K-Rowling"
        },
        "text": "\u201cIt is our choices, Harry, that show what we truly are, far more than our abilities.\u201d"
    }
];
    for (var i in data) {
        var d = data[i];
        var tags = $.map(d['tags'], function(t) {
            return "<a class='tag'>" + t + "</a>";
        }).join(" ");
        document.write("<div class='quote'><span class='text'>" + d['text'] + "</span><span>by <small class='author'>" + d['author']['name'] + "</small></span><div class='tags'>Tags: " + tags + "</div></div>");
        }
</script>
```
查看js代码，其`<div class="quote">`盒子的显示数据是通过for循环遍历数组data内的json渲染到页面中的。

In [ ]:
import requests
from pyquery import PyQuery as pq

url = 'https://movie.douban.com/subject_search?search_text=王祖贤&cat=1002'
response = requests.get(url)
doc = pq(response.text)
print('Covers:', doc('.cover').length)

import asyncio
from pyppeteer import launch
from pyquery import PyQuery as pq

async def main():
    browser = await launch()
    page = await browser.newPage()
    # await page.goto('http://quotes.toscrape.com/js/')
    await page.goto(url)

    doc = pq(await page.content())
    print('Covers:', doc('.cover').length)
    await browser.close()

asyncio.get_event_loop().run_until_complete(main())

```
Covers: 0
```
用pyppeteer抓取页面，发现没有任何结果。  
此方法对[豆瓣电影](https://movie.douban.com/subject_search?search_text=王祖贤&cat=1002)不适用。

## Selenium

当我们直接用 Requests 获取本页面HTML 的时候，发现想要的 XPath 并不存在。这是因为 HTML 还没有加载完，因此你需要一个工具，来进行网页加载的模拟，直到完成加载后再给你完整的 HTML。

在 Python 中，这个工具就是 Selenium 库，使用方法如下：
```
from selenium import webdriver
driver = webdriver.Chrome()
driver.get(request_url)
```

`Selenium` 是 Web 应用的测试工具，可以直接运行在浏览器中，它的原理是模拟用户在进行操作，支持当前多种主流的浏览器。

这里我们模拟 Chrome 浏览器的页面访问。

你需要先引用 Selenium 中的 WebDriver 库。WebDriver 实际上就是 Selenium 2，是一种用于 Web 应用程序的自动测试工具，提供了一套友好的 API，方便我们进行操作。

然后通过 WebDriver 创建一个 Chrome 浏览器的 drive，再通过 drive 获取访问页面的完整 HTML。

当你获取到完整的 HTML 时，就可以对 HTML 中的 XPath 进行提取，在这里我们需要找到图片地址 srcs 和电影名称 titles。这里通过 XPath 语法匹配到了多个元素，因为是多个元素，所以我们需要用 for 循环来对每个元素进行提取。

但 Selenium 用的时候有个麻烦事，就是环境的相关配置，得安装好相关浏览器，比如 Chrome、Firefox 等等，然后还要到官方网站去下载对应的驱动，最重要的还需要安装对应的 Python Selenium 库，确实是不是很方便，另外如果要做大规模部署的话，环境配置的一些问题也是个头疼的事情。

- Chrome
点击下载chrome的webdriver： http://chromedriver.storage.googleapis.com/index.html  
不同的Chrome的版本对应的chromedriver.exe 版本也不一样，下载时不要搞错了。如果是最新的Chrome, 下载最新的chromedriver.exe 就可以了。  
把chromedriver的路径也加到环境变量里。  
 
- Firefox
Firefox驱动下载地址为：https://github.com/mozilla/geckodriver/releases/  
根据自己的操作系统下载对应的驱动即可，使用的话，需要把驱动的路径和火狐浏览器的路径加入到环境变量里面才可以  
 
- IE
IE浏览器驱动下载地址为：http://selenium-release.storage.googleapis.com/index.html  
根据自己selenium版本下载对应版本的驱动即可，python的话，下载里面的IEDriverServerxxx.zip即可，这个是区分32和64位系统的，根据自己的系统下载即可，需要注意的是，如果要打开IE浏览器的话，需要在浏览器的Internet选项中的安全页里有4个安全选项，Internet、本地Internet、受信任的站点、受限制的站点，这4个里面都有一个启用保护模式，都需要勾选上才可以，还得把驱动的路径加入到环境变量中。

### Selenium使用
如果不想让Chrome页面启动，可以配置为headless模式，这样在启动的时候我们便看不到任何界面了。  

Tips：如果下载的chromedriver没有添加到环境变量里面，需要在`webdriver.Chrome()`中填写chromedriver路径。例如：`webdriver.Chrome(‘../chromedriver.exe’)`

- 添加环境变量
  - [Windows](https://blog.csdn.net/qq_41429288/article/details/80472064)
  - [Mac](https://blog.csdn.net/ywj_486/article/details/80940087)
  - Linux  
      在命令行中中进入下载文件所在路径，将其移动到/usr/bin  
      `sudo mv chromedriver /usr/bin`

运行之后会出现一个空白的 Chrom界面
![title](https://raw.githubusercontent.com/hufe09/GitNote-Images/master/gitnote/2019/05/14/1557764947071-1557764947079.png)
加载结束
![title](https://raw.githubusercontent.com/hufe09/GitNote-Images/master/gitnote/2019/05/14/1557764454475-1557764454500.png)


代码如下：

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from lxml import etree

def chrome_driver(url):
    driver = webdriver.Chrome()
    # 使用headless模式
#     driver = chrome_headless()
    # 请求地址
    driver.get(url)
    # 对当前屏幕截图
    driver.get_screenshot_as_file("screenshot.png")
    # 打印源页面
    # print(driver.page_source)
    html = etree.HTML(driver.page_source)
    # 关闭浏览器
    driver.close()
    # 退出webdriver
    driver.quit()
    return html

def chrome_headless():   
    chrome_options = Options()
    chrome_options.add_argument('--no-sandbox')
    # 使用headless模式：不打开浏览器
    chrome_options.add_argument('--headless')
    # 谷歌文档提到需要加上这个属性来规避bug
    chrome_options.add_argument('--disable-gpu')
    # 初始化实例
    driver = webdriver.Chrome(chrome_options=chrome_options)
    return driver
url = 'https://movie.douban.com/subject_search?search_text=王祖贤&cat=1002'
html = chrome_driver(url)
src_xpath = "//div[@class='item-root']/a[@class='cover-link']/img[@class='cover']/@src"
title_xpath = "//div[@class='item-root']/div[@class='detail']/div[@class='title']/a[@class='title-text']"
srcs = html.xpath(src_xpath)
titles = html.xpath(title_xpath)
print(f'Srcs:{len(srcs)}, Titles:{len(titles)}')

未使用headless模式，结果如下，爬取成功
```
Srcs:16, Titles:16
```
使用headless模式，结果为0，爬取失败
```
Srcs:0, Titles:0
```

完整爬取代码：

In [1]:
# cover_parse.py
import os
import time
from concurrent.futures import ThreadPoolExecutor, as_completed, wait, ALL_COMPLETED
from concurrent.futures import ProcessPoolExecutor

import requests
from lxml import etree
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


def download(src, img_name, query):
    """" 下载图片 """
    # print('启动下载进程，进程号[%d].' % os.getpid())
    dir = './' + query + '/' + str(img_name).strip('\u200e') + '.jpg'
    try:
        pic = requests.get(src, timeout=10)
        fp = open(dir, 'wb')
        fp.write(pic.content)
        fp.close()
        return '开始下载:' + str(img_name).strip('\u200e')
    except requests.exceptions.ConnectionError:
        return str(img_name) + '无法下载!'
    except OSError:
        return str(img_name) + '无法下载!'


def chrome_headless():
    chrome_options = Options()
    chrome_options.add_argument('--no-sandbox')
    # 使用headless模式：不打开浏览器
    chrome_options.add_argument('--headless')
    # 谷歌文档提到需要加上这个属性来规避bug
    chrome_options.add_argument('--disable-gpu')
    # 初始化实例
    driver = webdriver.Chrome(chrome_options=chrome_options)
    return driver


def chrome_driver(url):
    driver = webdriver.Chrome()
    # 使用headless模式
    # driver = chrome_headless()
    # 请求地址
    driver.get(url)
    # 对当前屏幕截图
    driver.get_screenshot_as_file("screenshot.png")
    # 打印源页面
    # print(driver.page_source)
    html = etree.HTML(driver.page_source)
    # 关闭浏览器
    driver.close()
    # 退出webdriver
    driver.quit()
    return html


def multi_threads_pool(num, func, lists):
    """线程池下载"""
    threads_pool = ThreadPoolExecutor(num)
    threads = [threads_pool.submit(lambda p: func(*p), list) for list in lists]
    # for t in threads:
    #     print(t.done())

    for future in as_completed(threads):
        # 使用as_completed方法一次取出所有任务的结果
        data = future.result()
        print(f"{data}")
    # wait方法可以让主线程阻塞，直到满足设定的要求
    wait(threads, return_when=ALL_COMPLETED)

def multi_process_pool(num, func, lists, query):
    '''进程池'''
    processes_pool = ProcessPoolExecutor(max_workers=num)
    # processes = [processes_pool.submit(lambda p: func(*p), list) for list in lists]
    processes = [processes_pool.submit(func, list, query) for list, query in zip(lists, [query] * len(lists))]
    for processe in as_completed(processes):
        # 使用as_completed方法一次取出所有任务的结果
        data = processe.result()
        print(f"{data}")
    # process_results = [task.result() for task in as_completed(processes)]
    # print(process_results)
    wait(processes, return_when=ALL_COMPLETED)


def single_thread(srcs, titles, query):
    i = 1
    for src, title in zip(srcs, titles):
        print(f'正在下载第{i}张图片')
        download(src, title.text, query)
        i += 1


def crawling_cover(url, query):
    # 获取html内容，方法1：
    # requests获取源网页，BeautifulSoup解析
    # response = requests.get(url)
    # html = BeautifulSoup(response.content, 'lxml')
    # srcs = html.find('.item-root .detail .cover src')
    # titles = html.find('.item-root .cover-link .title')

    # 获取html内容，方法2：Chrome Webdriver
    html = chrome_driver(url)

    # 解析html内容，方法2：Xpath
    src_xpath = "//div[@class='item-root']/a[@class='cover-link']/img[@class='cover']/@src"
    title_xpath = "//div[@class='item-root']/div[@class='detail']/div[@class='title']/a[@class='title-text']"
    srcs = html.xpath(src_xpath)
    titles = html.xpath(title_xpath)

    lists = []
    for src, title, q in zip(srcs, titles, [query] * len(srcs)):
        print(src, title.text)
        lists.append([src, title.text, q])

    start = time.time()
    print('启动下载进程，进程号[%d].' % os.getpid())
    # 线程池下载
    multi_threads_pool(8, download, lists)

    # 单线程下载
    # single_thread(srcs, titles, query)

    end = time.time()
    print('本次下载耗时%.2f秒.' % (end - start))
    
    values = url.split('?')[- 1]
    gets = {}
    for key_value in values.split('&'):
        list = key_value.split('=')
        gets[list[0]] = list[1]
    if (gets.__contains__('start')):
        page = int(gets['start']) // 15 + 1
        pages = '------------------------已爬取第'+ str(page) +'页----------------------------------'
    else:
        pages = '------------------------已爬取第1页----------------------------------'
    return pages

def main():

    query = '王祖贤'
    # 创建文件夹
    folder_name = query
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    urls = []
    for i in range(0, 15 * 2, 15):
        if i == 0:
            urls.append("https://movie.douban.com/subject_search?search_text=" + query + "&cat=1002")
        else:
            urls.append("https://movie.douban.com/subject_search?search_text=" + query + "&cat=1002&start=" + str(i))

    lists = []
    for url, q in zip(urls, [query] * len(urls)):
        lists.append([url, q])
    start = time.time()
    
    #使用进程池，进程数为2
    #multi_process_pool(2, crawling_cover, urls, query)
    
    #单线程
    for url in urls:
        print(crawling_cover(url, query))
    end = time.time()
    print(f'Download Finished!\n耗时{end - start}秒')


if __name__ == '__main__':
    main()

WebDriverException: Message: unknown error: Chrome failed to start: exited abnormally
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Linux 4.15.0-29-generic x86_64)


![title](https://raw.githubusercontent.com/hufe09/GitNote-Images/master/gitnote/2019/05/14/1557764874462-1557764874485.png)